# Project 1 Solution: Discussion

## MCS 275 Spring 2021 - Emily Dumas

## What this document contains

A set of solutions to Project 1 was released on the course web site in the form of a module `tasks.py`.

This document contains the entire source of the solutions, but with occasional breaks to discuss how the solution works, or other methods that might be considered.

**Note:** Because the source code of the solution is split between multiple cells, it can't be used directly in this notebook.  If you want a version you can actually test, download the solution file `tasks.py` from the course web page.

## Solution code with commentary

In [ ]:
"Task classes for MCS 275 Spring 2021 Project 1"
# There is also a document containing discussion of
# this solution.  Please read that for more info.
# Emily Dumas


class Task:
    "Base class for all task types"
    def __init__(self, description):
        "Initialize new Task"
        self.description = description
        self.active = False

    def next_time(self):
        "Return next time task must run (not supported in this base class)"
        raise NotImplementedError(
            "Task not intended to be instantiated directly"
        )
    

The method `next_time` was required to raise an exception.  Any exception type was acceptable, but since Python has a built-in exception type reserved for methods that are not supposed to be called, `NotImplementedError`, we use that one.  It would be equally valid to replace every instance of `NotImplementedError` with `Exception`.

In [ ]:
    def run(self):
        "Run the task"
        raise NotImplementedError(
            "Task not intended to be instantiated directly"
        )

    def retire(self, t):
        "Retire task instances up to time t"
        raise NotImplementedError(
            "Task not intended to be instantiated directly"
        )


class OneTimeTask(Task):
    "Task that runs once at a designated time"
    def __init__(self, description, t):
        """Initialize a new OneTimeTask with description `description`
        and run time `t`"""
        super().__init__(description)
        self.active = True
        self.t = t
    

It's probably worth pointing out at this point that in the constructor of `OneTimeTask`, attributes named `description` and `active` must be created.  However, the project description allows us to choose how to store the task's scheduled time (given as constructor argument `t`).  We choose to store it in an instance attribute also named `t`, but we could also have used a different name.

In [ ]:
    def next_time(self):
        "Return next time task must run"
        if not self.active:
            raise Exception("Task is not active")
        return self.t

This method is meant to be run, but may need to signal a problem that prevents it from returning a value.  Using `Exception` or a custom subclass meant for this purpose is the best option.  The `NotImplementedError` returned by some methods of `Task` and `RecurringTask` would not be appropriate here.  (However, *any* exception type was deemed acceptable in this assignment.)

In [ ]:
    def run(self):
        "Run the task"
        print("run: class={} description=\"{}\" time={}".format(
            self.__class__.__name__,
            self.description,
            self.t
        ))

Using `self.__class__.__name__` here is a way to ensure the `run` method always prints a message that uses the current class name, even if this method is inherited by a subclass.  It isn't strictly necessary, because the project specifications do not involve `run` being inherited by any of the classes.

In [ ]:
    def retire(self, t):
        "Retire task if its running time is less than or equal to `t`"
        if t >= self.t:
            self.active = False


class RecurringTask(Task):
    "Base class for tasks that run more than once"
    def num_until(self, end):
        "How many times will the task run before time `end`?"
        raise NotImplementedError(
            "RecurringTask not intended to be instantiated directly"
        )


class BoundedRecurringTask(RecurringTask):
    "Task that runs at times in a finite arithmetic progression"
    def __init__(self, description, start, gap, n):
        """Initialize a recurring task that runs first at time `start`, again at `start`+`gap`, 
        and so on, until it has run `n` times"""
        super().__init__(description)
        self.t_next = start
        self.gap = gap
        self.times_left = n
        self.active = self.times_left > 0

Here we choose to store `start` and `n` in instance attributes, but to give them slightly different names to reflect their ongoing role as the task execution proceeds.  We've decided to keep track of how many times the task still needs to run, and the next time it will run.  Each time an instance of the task is retied, `t_next` and `times_left` will be updated accordingly.

Another approach would be to store a list of all the times the task will run, for example in an instance attribute `times`.  Then, the `retire` method would simply remove certain elements from that list, and `next_time` would return the first element of the list of times.

However, while perfectly valid for this assignment, the approach based on a list of all times has two deficiencies:
* It would make the implementation of `UnboundedRecurringTask` significantly different, since that class cannot store all of its scheduled times, and
* It makes the expense of creating a `BoundedRecurringTask` (measured in time or memory) proportional to `n`, which is larger than necessary.

In [ ]:
    def run(self):
        "Run the task"
        if not self.active:
            raise Exception("task is not active")
        print("run: class={} description=\"{}\" time={}".format(
            self.__class__.__name__,
            self.description,
            self.t_next
        ))

    def next_time(self):
        "Return next time task must run"
        if self.active:
            return self.t_next
        else:
            raise Exception("task is not active")

    def num_until(self, end):
        "How many times will the task run before time `end`?"
        if end < self.t_next:
            # The next instance is after `end`, so the answer is zero
            return 0
        # How many instances if the task ran forever
        n = 1 + ((end-self.t_next) // self.gap)
        # Actual number is clipped by self.times_left
        return min(n,self.times_left)

The mathematical expressions in `num_until` deserve some explanation.

First, `end - self.t_next` is the number of units of time between the next task run time and the endpoint specified as an argument to `num_until`.  If this is less than `self.gap`, then of course the ask will only run once before `end`.  More generally, we need to know how many intervals of size `self.gap` can fit into the period between `self.t_next` and `end`, which is given by integer division.  That quotient indicates how many *additional* times the task will run, after the first one.  Hence we add one to get the actual number to be retired.

So far, this logic hasn't considered the limit on the total number of times the task needs to run, so the last line of the function reduces the answer to `self.times_left` if it is too large.

A number of students used an expression like
```python
int((end-self.t_next) / self.gap)
```
instead of
```python
(end-self.t_next) / self.gap
```
For moderate values of `end`, this will work.  However, these are not exactly equivalent.  Using true division (`/`) produces a float, and `int()` converts it back to an integer.  In Python, integers have no preset limit on their size, but floats do, so if `end-self.t_next` were an absolutely huge multiple of `self.gap`, the version with true division would fail.  For example, try these two expressions in the REPL:
```python
10**1000 / 2   # Fails; 10 to the power 1000 too large for a float
10**1000 // 2  # Succeeds
```
In this project, it was indicated that time is an integer, so there is no need to introduce artificial limitations on its size.  (There was no grade penalty for use of true division, however.)

In [ ]:
    def num_total(self):
        "How many times will the task run (excluding retired instances)?"
        return self.times_left

    def retire(self, t):
        "Retire instances of the task up to and including time `t`"
        k = self.num_until(t)
        self.t_next += k*self.gap
        self.times_left -= k
        self.active = self.times_left > 0

Notice how having `retire` call `num_until` is a convenient way to find out how many instances need to be retired, significantly simplifying the math in this method.

In [ ]:
class UnboundedRecurringTask(RecurringTask):
    "Task that runs at times in an infinite arithmetic progression"
    def __init__(self, description, start, gap):
        """Initialize a recurring task that runs first at time `start`, again at `start+gap`, 
        `start+2*gap`, and so on, forever."""
        super().__init__(description)
        self.t_next = start
        self.gap = gap
        self.active = True

    def run(self):
        "Run the task"
        print("run: class={} description=\"{}\" time={}".format(
            self.__class__.__name__,
            self.description,
            self.t_next
        ))

    def next_time(self):
        "Return next time task must run"
        return self.t_next

    def num_until(self, end):
        "How many times will the task run before time `end`?"
        if end < self.t_next:
            # The next instance is after `end`, so the answer is zero
            return 0
        return 1 + ((end-self.t_next) // self.gap)

This is just like `BoundedRecurringTask.num_until` except without the need to check a limit on the total number of times it will run.

In [ ]:
    def retire(self, t):
        "Retire instances of the task up to and including time `t`"
        k = self.num_until(t)
        self.t_next += k*self.gap

Here again, calling `num_until` inside `retire` makes it much simpler.

## Revision History

* 2021-02-09 Initial publication